In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#| code-fold: false
# IMPORTING LIBRARIES
# --------------------

# AstroML
from astroML.datasets import fetch_LINEAR_sample
from astropy.coordinates import SkyCoord
import astropy.units as u

# Basic libraries
import pickle
import os
import sys
from tqdm import tqdm

# Plotting
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.font_manager import FontProperties

# DataFrame analysis
import pandas as pd

# Math libraries
import numpy as np
import scipy as sc
from scipy.stats import norm

# CONFIGURATION
# -------------
sns.set_theme(style='white') # setting the theme for plotting
sys.path.insert(0,'../src/')
np.random.seed(42)

# configuring plotting colors
colors = ['#465BBB', '#A82F43', '#3F8FCE', '#7ABBCE', '#3A3865', '#612A37', '#DC5433', '#F29457']
b1 = '#465BBB'
b2 = '#3F8FCE'
b3 = '#7ABBCE'
b4 = '#3A3865'
black1 = '#22212A'
black2 = '#2D1E21'
or1 = '#A82F43'
or2 = '#612A37'
or3 = '#DC5433'
or4 = '#F29457'
muted_colors = sns.set_palette(sns.color_palette(colors))

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] = r'\usepackage{txfonts}'  # Use TX Times font

# Set up your font properties (optional)
font = FontProperties()
font.set_family('serif')
font.set_name('Times')  # Times font
%matplotlib inline 

In [4]:
#| code-fold: false
# Importing custom libraries
# ----------------------------
sys.path.insert(0,'../src/')
from BE_plotting import*
#from RR_lyrae_params import*
from blazhko_analysis import*

In [18]:
#| code-fold: false
# DATA
dataL = fetch_LINEAR_sample(data_home='../inputs') # fetching the data from astroML data library
end = 'ffinal'

# DATA from previous notebook
BE_STARS = pd.read_csv("../outputs/FINAL_BE_CATALOGUE_EXPANDED.csv")
be_cand = pd.read_csv('../outputs/blazhko_candidates_expanded.csv')
lc_analysis = pd.read_csv("../outputs/LC_ANALYSIS_"+end+".csv")
rrlyrae = pd.read_csv('../outputs/Lrrlyr_unprocessed.csv')
periodogr = pickle.load(open("../outputs/periodograms_"+end+".pkl", "rb"))
fits = np.load("../outputs/fits_"+end+".npy", allow_pickle=True)
ztf_data = np.load("../outputs/ztf_data_filter"+end+".npy", allow_pickle=True)

---

# Creating a master table

In [19]:
lc_analysis.head()

,LINEAR id,Plinear,Pztf,Pmean,Pratio,NdataLINEAR,L_chi2dof,L_chi2dofR,Lmean_chi2dof,Lmean_chi2dofR,...,MainPeakZ,BlazhkoPeakZ,BlazhkoPeriodZ,BpowerRatioZ,BsignificanceZ,Ampl_diff,dP,IndicatorType,ChiType,BE_score
0,29848,0.557020,0.557040,0.557030,1.000036,301,3.0,1.4,3.3,1.6,...,1.7952,1.7982,333.3333,0.2573,10.0729,0.37,0.00004,NaN,NaN,NaN
1,50402,0.643303,0.643294,0.643298,0.999986,284,0.6,0.7,0.6,0.7,...,1.5545,1.5918,26.8420,0.0027,8.9208,0.21,0.00001,NaN,NaN,NaN
2,62892,0.530776,0.530785,0.530780,1.000017,276,1.1,0.9,1.1,1.0,...,1.8840,1.9433,16.8634,0.0048,13.1609,0.02,0.00002,NaN,NaN,NaN
3,91437,0.674733,0.674737,0.674735,1.000006,177,2.8,1.3,2.8,1.3,...,1.4821,1.4849,355.8719,0.0233,16.1566,0.34,0.00001,NaN,NaN,NaN
4,95250,0.313870,0.313876,0.313873,1.000019,222,0.8,0.8,0.9,0.8,...,3.1860,3.1889,342.4658,0.0028,13.4528,0.02,0.00002,NaN,NaN,NaN


In [20]:
lc_MASTER = pd.DataFrame(())

In [21]:
lc_MASTER['LINEAR ID'] = lc_analysis['LINEAR id']
lc_MASTER['Plinear'] = lc_analysis['Plinear']
lc_MASTER['Pztf'] = lc_analysis['Pztf']
lc_MASTER['N_L'] = lc_analysis['NdataLINEAR']
lc_MASTER['N_Z'] = lc_analysis['NdataZTF']
lc_MASTER['L_chi2r'] = lc_analysis['L_chi2dofR']
lc_MASTER['Z_chi2r'] = lc_analysis['Zchi2dofR']
lc_MASTER['L_chi2'] = lc_analysis['L_chi2dof']
lc_MASTER['Z_chi2'] = lc_analysis['Zchi2dof']
lc_MASTER['Lampl'] = lc_analysis['Lampl']
lc_MASTER['Zampl'] = lc_analysis['Zampl']
lc_MASTER['Ampl_diff'] = lc_analysis['Ampl_diff']
lc_MASTER['BpeakL'] = lc_analysis['BlazhkoPeakL']
lc_MASTER['BpeakZ'] = lc_analysis['BlazhkoPeakZ']
lc_MASTER['BperiodL'] = lc_analysis['BlazhkoPeriodL']
lc_MASTER['BperiodZ'] = lc_analysis['BlazhkoPeriodZ']
lc_MASTER['Periodogram_f'] = '-'
lc_MASTER['B_score'] = 0  # Assuming a default value of 0 for B_score
lc_MASTER['Blazhko_f'] = '-1'  # Default value

In [22]:
lc_MASTER.to_csv('../outputs/lc_MASTER_plain.csv', index=False)

In [23]:
lcLids = list(lc_MASTER['LINEAR ID'])
bcLids = list(be_cand['LINEAR id'])
bLids = list(BE_STARS['LINEAR id'])

In [24]:
lc_MASTER.head()

,LINEAR ID,Plinear,Pztf,N_L,N_Z,L_chi2r,Z_chi2r,L_chi2,Z_chi2,Lampl,Zampl,Ampl_diff,BpeakL,BpeakZ,BperiodL,BperiodZ,Periodogram_f,B_score,Blazhko_f
0,29848,0.557020,0.557040,301,43,1.4,3.5,3.0,12.6,0.56,0.93,0.37,1.8328,1.7982,26.6205,333.3333,-,0,-1
1,50402,0.643303,0.643294,284,586,0.7,1.1,0.6,1.8,0.48,0.69,0.21,1.6223,1.5918,14.7351,26.8420,-,0,-1
2,62892,0.530776,0.530785,276,771,0.9,3.2,1.1,19.8,0.62,0.64,0.02,1.9519,1.9433,14.7319,16.8634,-,0,-1
3,91437,0.674733,0.674737,177,564,1.3,2.0,2.8,5.6,0.87,1.21,0.34,1.5498,1.4849,14.7580,355.8719,-,0,-1
4,95250,0.313870,0.313876,222,916,0.8,1.4,0.8,3.0,0.48,0.46,0.02,3.2565,3.1889,14.1844,342.4658,-,0,-1


In [25]:
for n, i in enumerate(lcLids):
    for m, j in enumerate(bcLids):
        if i == j:
            if be_cand.loc[m, 'BE_score'] > 0:
                lc_MASTER.loc[n, 'B_score'] = be_cand.loc[m, 'BE_score']
            for nm, k in enumerate(bLids):
                if i == k:
                    if BE_STARS.loc[nm, 'IndicatorType'] == 'Z' or BE_STARS.loc[nm, 'IndicatorType'] == 'L' or BE_STARS.loc[nm, 'IndicatorType'] == 'LZ':
                        lc_MASTER.loc[n, 'Periodogram_f'] = BE_STARS.loc[nm, 'IndicatorType']
                        lc_MASTER.loc[n, 'Blazhko_f'] = '2'
                    else:
                        lc_MASTER.loc[n, 'Blazhko_f'] = '1'
                    break  
                else:
                    lc_MASTER.loc[n, 'Blazhko_f'] = '0'
            break 

In [26]:
lc_MASTER.head()

,LINEAR ID,Plinear,Pztf,N_L,N_Z,L_chi2r,Z_chi2r,L_chi2,Z_chi2,Lampl,Zampl,Ampl_diff,BpeakL,BpeakZ,BperiodL,BperiodZ,Periodogram_f,B_score,Blazhko_f
0,29848,0.557020,0.557040,301,43,1.4,3.5,3.0,12.6,0.56,0.93,0.37,1.8328,1.7982,26.6205,333.3333,-,6,1
1,50402,0.643303,0.643294,284,586,0.7,1.1,0.6,1.8,0.48,0.69,0.21,1.6223,1.5918,14.7351,26.8420,-,0,-1
2,62892,0.530776,0.530785,276,771,0.9,3.2,1.1,19.8,0.62,0.64,0.02,1.9519,1.9433,14.7319,16.8634,-,5,1
3,91437,0.674733,0.674737,177,564,1.3,2.0,2.8,5.6,0.87,1.21,0.34,1.5498,1.4849,14.7580,355.8719,-,0,-1
4,95250,0.313870,0.313876,222,916,0.8,1.4,0.8,3.0,0.48,0.46,0.02,3.2565,3.1889,14.1844,342.4658,-,0,-1


---

In [27]:
lc_MASTER.to_csv("../outputs/lc_MASTER.csv", index=False)

---

In [28]:
lc_MASTER = pd.read_csv('../outputs/lc_MASTER.csv')

In [29]:
with open('../PAPER/lc_master.tex', "w") as f:
    f.write(lc_MASTER.to_latex(index=False))

/var/folders/t5/p2kd2rtd7bz4dz4p833qfm3w0000gn/T/ipykernel_1079/2370270825.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(lc_MASTER.to_latex(index=False))
